# A. State preparation with Jaynes-Cummings controls

In [1]:
# ruff: noqa
import os

os.sys.path.append("../../../..")

In [2]:
from feedback_grape.fgrape import optimize_pulse_with_feedback
from feedback_grape.utils.operators import (
    sigmap,
    sigmam,
    create,
    destroy,
    identity,
)
from feedback_grape.utils.states import basis, fock
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp
from jax.scipy.linalg import expm

As a preliminary step, we consider state preparation
of a target state starting from a pure state. In addition,
we assume that any coupling to an external environment
is negligible and that the parametrized controls can be
implemented perfectly.

Here no feedback is required, we are just testing the parameterized gates setup.

As a first example, we consider the state preparation
of a cavity resonantly coupled to an externally driven
qubit

Here, we consider a particular sequence of
parametrized unitary gates originally introduced by Law
and Eberly

In [3]:
N_cav = 30

In [4]:
def qubit_unitary(alphas):
    alpha_re = alphas[0]
    alpha_im = alphas[1]
    alpha = alpha_re + 1j * alpha_im
    return tensor(
        identity(N_cav),
        expm(-1j * (alpha * sigmap() + alpha.conjugate() * sigmam()) / 2),
    )

In [5]:
def qubit_cavity_unitary(beta_re):
    beta = beta_re
    return expm(
        -1j
        * (
            beta * (tensor(destroy(N_cav), sigmap()))
            + beta.conjugate() * (tensor(create(N_cav), sigmam()))
        )
        / 2
    )

In their groundbreaking work, Law and Eberly have
shown that any arbitrary superposition of Fock states with
maximal excitation number N can be prepared out of the
ground state in a sequence of N such interleaved gates,
also providing an algorithm to find the correct angles and
interaction durations

## First target is the state $ | 1 ⟩ + | 3 ⟩ $ 

In [6]:
# TODO: Why if time steps are less than 5 then it plateus at 0.5
time_steps = 5  # corressponds to maximal excitation number of an arbitrary Fock State Superposition

In [7]:
psi0 = tensor(basis(N_cav), basis(2))
psi0 = psi0 / jnp.linalg.norm(psi0)
psi_target = tensor((fock(N_cav, 1) + fock(N_cav, 3)) / jnp.sqrt(2), basis(2))
psi_target = psi_target / jnp.linalg.norm(psi_target)

In [8]:
psi0.shape

(60, 1)

In [9]:
psi_target.shape

(60, 1)

In [10]:
print(fock(N_cav, 1))

[[0.+0.j]
 [1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]


Law and Eberly provided an algorithm to determine the correct parameters for state preparation. These include:

- The rotation angle $ |\alpha| $,
- The azimuthal angle $ \arg\left(\frac{\alpha}{|\alpha|}\right) $,
- The interaction duration $ |\beta| $. <br>

So Goal is to find the best control vector (rather than control amplitudes, this time) that leads to finding the optimal state-preparation strategies. Performing as well as the Law-Eberly algorithm.

## Optimizing
Currently l-bfgs with the same learning rate of 0.3 converges at a local minimum of 0.5, adam also converges at 0.5 but at smaller learning rates

In [ ]:
from feedback_grape.fgrape import Gate
import jax

key = jax.random.PRNGKey(42)
# not provideing param_constraints just propagates the same initial_parameters for each time step
qub_unitary = Gate(
    gate=qubit_unitary,
    initial_params=jax.random.uniform(
        key,
        shape=(2,),  # 2 for gamma and delta
        minval=-jnp.pi,
        maxval=jnp.pi,
        dtype=jnp.float64,
    ),
    measurement_flag=False,
    param_constraints=[
        [-2 * jnp.pi, 2 * jnp.pi],
        [-2 * jnp.pi, 2 * jnp.pi],
    ],
)

qub_cav = Gate(
    gate=qubit_cavity_unitary,
    initial_params=jax.random.uniform(
        key,
        shape=(1,),
        minval=-jnp.pi,
        maxval=jnp.pi,
        dtype=jnp.float64,
    ),
    measurement_flag=False,
    param_constraints=[[-2 * jnp.pi, 2 * jnp.pi]],
)

system_params = [qub_unitary, qub_cav]


result = optimize_pulse_with_feedback(
    U_0=psi0,
    C_target=psi_target,
    system_params=system_params,
    num_time_steps=time_steps,
    max_iter=1000,
    convergence_threshold=1e-16,
    evo_type="state",
    mode="no-measurement",
    goal="fidelity",
    learning_rate=0.02,
    batch_size=10,
    eval_batch_size=2,
)

In [12]:
len(result.returned_params)

5

In [13]:
# here makes sense for each batch size we have a different set of parameters since there are no measurements and therefore no stochasticisty or randomness
result.returned_params

[[Array([[ 0.56028059, -6.32020193],
         [ 0.56028059, -6.32020193]], dtype=float64),
  Array([[-3.56962321],
         [-3.56962321]], dtype=float64)],
 [Array([[-7.99477043e-07,  3.14100353e+00],
         [-7.99477043e-07,  3.14100353e+00]], dtype=float64),
  Array([[2.77362796],
         [2.77362796]], dtype=float64)],
 [Array([[3.14147369e+00, 3.76765044e-07],
         [3.14147369e+00, 3.76765044e-07]], dtype=float64),
  Array([[-1.57658746],
         [-1.57658746]], dtype=float64)],
 [Array([[0.54369561, 6.38184408],
         [0.54369561, 6.38184408]], dtype=float64),
  Array([[2.61430116],
         [2.61430116]], dtype=float64)],
 [Array([[ 3.64288481e-07, -3.14422887e+00],
         [ 3.64288481e-07, -3.14422887e+00]], dtype=float64),
  Array([[3.14159383],
         [3.14159383]], dtype=float64)]]

In [14]:
print(result.final_state)

[[[ 4.19984315e-05-1.10385769e-06j]
  [-8.42375825e-08-5.28111620e-08j]
  [-7.07106588e-01-9.98539451e-08j]
  [ 1.40454368e-05-4.90337326e-05j]
  [ 1.34493825e-04+2.79463836e-05j]
  [-3.17259534e-08-2.08223061e-07j]
  [-7.07106764e-01-1.22412466e-07j]
  [-6.04977148e-05-2.42910434e-05j]
  [-4.18544291e-05-4.82056872e-04j]
  [-5.91921616e-08+1.52059890e-07j]
  [ 0.00000000e+00+0.00000000e+00j]
  [-1.87424797e-04+1.62730962e-05j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.0000000

In [15]:
print(result)

FgResult(optimized_trainable_parameters=[[Array([ 0.56028059, -6.32020193], dtype=float64), Array([-3.56962321], dtype=float64)], [Array([-7.99477043e-07,  3.14100353e+00], dtype=float64), Array([2.77362796], dtype=float64)], [Array([3.14147369e+00, 3.76765044e-07], dtype=float64), Array([-1.57658746], dtype=float64)], [Array([0.54369561, 6.38184408], dtype=float64), Array([2.61430116], dtype=float64)], [Array([ 3.64288481e-07, -3.14422887e+00], dtype=float64), Array([3.14159383], dtype=float64)]], iterations=1000, final_state=Array([[[ 4.19984315e-05-1.10385769e-06j],
        [-8.42375825e-08-5.28111620e-08j],
        [-7.07106588e-01-9.98539451e-08j],
        [ 1.40454368e-05-4.90337326e-05j],
        [ 1.34493825e-04+2.79463836e-05j],
        [-3.17259534e-08-2.08223061e-07j],
        [-7.07106764e-01-1.22412466e-07j],
        [-6.04977148e-05-2.42910434e-05j],
        [-4.18544291e-05-4.82056872e-04j],
        [-5.91921616e-08+1.52059890e-07j],
        [ 0.00000000e+00+0.00000000e+

In [16]:
result.final_fidelity

Array(0.9999997, dtype=float64)